<a href="https://colab.research.google.com/github/Riju-88/tmp_files/blob/main/colab/Yue_extend_with_exllama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab for https://github.com/Mozer/YuE-extend
- It can extend any uploaded mp3
- uses exllama2 module
- flash_attention 2 is not supported for Tesla T4

# Install

In [1]:
# Step 1: Install ExLlamaV2
!pip install exllamav2

# Step 2: Clone the YuE-extend repo
!git clone https://github.com/Mozer/YuE-extend
%cd YuE-extend

# Step 3: Uninstall conflicting packages before installing new ones
!pip uninstall -y protobuf google-cloud google-cloud-storage google-auth flash-attn

# Step 4: Install dependencies
!pip install -r requirements.txt

# Step 5: Clone the Hugging Face model
!git clone https://huggingface.co/m-a-p/xcodec_mini_infer

# Step 6: Install specific version of protobuf to avoid import errors
!pip install protobuf==3.20.3

# Step 7: Upgrade necessary packages
!pip install --upgrade transformers tensorflow torch torchvision torchaudio


fatal: destination path 'YuE-extend' already exists and is not an empty directory.
/content/YuE-extend
Found existing installation: protobuf 3.20.3
Uninstalling protobuf-3.20.3:
  Successfully uninstalled protobuf-3.20.3
Found existing installation: flash_attn 2.7.4.post1
Uninstalling flash_attn-2.7.4.post1:
  Successfully uninstalled flash_attn-2.7.4.post1
  Using cached flash_attn-2.7.4.post1-cp311-cp311-linux_x86_64.whl
  Using cached protobuf-3.19.6-py2.py3-none-any.whl.metadata (828 bytes)
Using cached protobuf-3.19.6-py2.py3-none-any.whl (162 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.84.0 requires google-auth<3.0.0dev,>=2.14.1, which is not installed.
google-cloud-aiplatform 1.84.0 requires google-cloud-storage<3.0.0dev,>=1.32.0, which is not installed.
google-api-core 2.24.2 requires google-auth<3.0.0,>=2.14.1, which i

fatal: destination path 'xcodec_mini_infer' already exists and is not an empty directory.
  Using cached protobuf-3.20.3-py2.py3-none-any.whl.metadata (720 bytes)
Using cached protobuf-3.20.3-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-aiplatform 1.84.0 requires google-auth<3.0.0dev,>=2.14.1, which is not installed.
google-cloud-aiplatform 1.84.0 requires google-cloud-storage<3.0.0dev,>=1.32.0, which is not installed.
google-api-core 2.24.2 requires google-auth<3.0.0,>=2.14.1, which is not installed.
google-api-python-client 2.164.0 requires google-auth!=2.24.0,!=2.25.0,<3.0.0.dev0,>=1.32.0, which is not installed.
google-cloud-firestore 2.20.1 requires g

# Download models

In [2]:
# download COT s1 + s2 models
# you can also download ICL model (optional)
!huggingface-cli download Alissonerdx/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw --local-dir workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw
!huggingface-cli download Alissonerdx/YuE-s2-1B-general-exl2-8.0bpw --local-dir workspace/models/YuE-s2-1B-general-exl2-8.0bpw

Fetching 7 files: 100% 7/7 [00:00<00:00, 25822.45it/s]
/content/YuE-extend/workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw
Fetching 6 files: 100% 6/6 [00:00<00:00, 1609.58it/s]
/content/YuE-extend/workspace/models/YuE-s2-1B-general-exl2-8.0bpw


Don't start UI. Google colab doesn't like UIs. Use command line instead. You can edit lyrics and genre in Files on the left -> Yue-extend/prompt_egs/lyrics.txt (double click). Output mp3 will be saved in workspace/outputs/.

# lyrics2music

In [ ]:
# simple cot music generation. 1 verse (run_n_segments), 30s (max_new_tokens)
!python -u src/yue/infer.py --stage1_use_exl2 --stage1_model "workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw" --stage1_cache_size 30000 --stage1_cache_mode FP16 --stage2_use_exl2 --stage2_model "workspace/models/YuE-s2-1B-general-exl2-8.0bpw" --stage2_cache_size 50000 --stage2_cache_mode 'FP16' --genre_txt "prompt_egs/genre.txt" --lyrics_txt "prompt_egs/lyrics.txt" --run_n_segments 1 --output_dir workspace/outputs --cuda_idx 0 --seed 42 --max_new_tokens 3000 --basic_model_config xcodec_mini_infer/final_ckpt/config.yaml --resume_path xcodec_mini_infer/final_ckpt/ckpt_00360000.pth --config_path xcodec_mini_infer/decoders/config.yaml --vocal_decoder_path xcodec_mini_infer/decoders/decoder_131000.pth --inst_decoder_path xcodec_mini_infer/decoders/decoder_151000.pth --resume_after_n -1

Loading exllamav2_ext extension (JIT)...


# mp3 extend

Trim your mp3 up to 30s. Use "Files" menu on the left to upload your split pop.00001.Vocals.mp3 + pop.00001.Instrumental.mp3 to Yue-extend/prompt_egs. To split use: https://huggingface.co/spaces/theneos/audio-separator or other services.
Don't forget to update genre.txt and put full lyrics into lyrics.txt

In [ ]:
# mp3 extend cot. 1 verse (run_n_segments), 28s (max_new_tokens), 7s from mp3 (extend_mp3_end_time)
!python -u src/yue/infer.py --stage1_use_exl2 --stage1_model "workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw" --stage1_cache_size 25000 --stage1_cache_mode FP16 --stage2_use_exl2 --stage2_model "workspace/models/YuE-s2-1B-general-exl2-8.0bpw" --stage2_cache_size 50000 --stage2_cache_mode 'FP16' --genre_txt "prompt_egs/genre.txt" --lyrics_txt "prompt_egs/lyrics.txt" --run_n_segments 1 --output_dir workspace/outputs --cuda_idx 0 --seed 42 --max_new_tokens 2800 --basic_model_config xcodec_mini_infer/final_ckpt/config.yaml --resume_path xcodec_mini_infer/final_ckpt/ckpt_00360000.pth --config_path xcodec_mini_infer/decoders/config.yaml --vocal_decoder_path xcodec_mini_infer/decoders/decoder_131000.pth --inst_decoder_path xcodec_mini_infer/decoders/decoder_151000.pth --extend_mp3 --vocal_track_prompt_path "prompt_egs/pop.00001.Vocals.mp3" --instrumental_track_prompt_path "prompt_egs/pop.00001.Instrumental.mp3" --extend_mp3_end_time 7 --prompt_start_time 0 --prompt_end_time 8 --resume_after_n -1